### *Import Packages and Modules*
***

In [1]:
import xarray as xr
import numpy as np
import netCDF4 as nc
import glob
import re
import matplotlib.pyplot as plt
import matplotlib
import warnings
from matplotlib import *
from netCDF4 import Dataset
from scipy import stats
from matplotlib.colors import LinearSegmentedColormap
from pylr2 import regress2
import itertools
import statsmodels.api as sm
import time
from operator import itemgetter
import csv
import statistics
from major_axis_regression import regress2
from parula import parula
# parula is a .py file that should be found in script that adds parula, the same colour scale as Cael uses to the data. 

In [2]:
parula_map = parula()

***
### *Import Data*


In [3]:
forjoesouth = np.loadtxt(open("../data/forjoesouth.csv"), delimiter=",") # imports Cael's data for the Southern Ocean

In [4]:
forjoesouth.shape

(768, 6)

In [5]:
# lists which indexes are which variables
bb_depths_so = forjoesouth[:,0]
bb_Foc_so = forjoesouth[:,3]
bb_Fic_so = forjoesouth[:,4]
bb_Fsi_so = forjoesouth[:,5]

------
### Define Function

In [6]:
def JG_BBCael(data,bbeta,data_no,bbeta_pair_no):
    import statsmodels.api as sm
    import numpy as np 

    # find random coordinate points for SO 

    index_number = np.random.randint(0, high=768, size=data_no)        
    
    #print('average in mask:', np.average(new_mask))
    # this average should be 1 == only areas with values are selected
    #print('(should be one or else something went wrong)')

    # use randomly selected coordinate points to select 10000 random values of Foc, Fic and Fsi 
    random_depth = data[index_number,0]
    random_Foc = data[index_number, 3]
    random_Fic = data[index_number, 4]
    random_Fsi = data[index_number, 5]
    
    #y = np.power((random_depth[:, np.newaxis]/1000), bbeta[:, 0])
    #y = np.log((random_Foc[:, np.newaxis] * y))
    #x = np.log((random_Fic[:, np.newaxis]) + (bbeta[:, 1] * random_Fsi[:, np.newaxis]))

    y = np.log(random_Foc[:, np.newaxis]) + (bbeta[:, 0] * np.subtract(np.log(random_depth[:, np.newaxis]),np.log(1000)))
    x = np.log((random_Fic[:, np.newaxis]) + (bbeta[:, 1] * random_Fsi[:, np.newaxis]))

    
    r_squared = np.ones((bbeta_pair_no))*np.nan
    slope = np.ones((bbeta_pair_no))*np.nan
    intercept = np.ones((bbeta_pair_no))*np.nan

    for i in np.arange(0,bbeta_pair_no,1):
        r_squared[i], slope[i], intercept[i] = regress2(x[:,i], y[:,i], _method_type_2="major axis")

    r_maximum = np.nanmax(r_squared)

    where_max = np.where(r_squared==r_maximum)

    b_maxr = np.nanmean(bbeta[where_max, 0])
    beta_maxr = np.nanmean(bbeta[where_max, 1])
    slope_max = np.nanmean(slope[where_max])
    intercept_max = np.nanmean(intercept[where_max])

    return r_maximum, b_maxr, beta_maxr, slope_max, intercept_max


------
### Create the b and beta range

In [7]:
# then we create b and beta range to search for in each sample taken. 
# these are estimated from a small b and beta sample. 
# later we ensure that the data isn't skewed to one side of the distribution searched within b and beta. 

In [8]:
# define b and beta
b = numpy.arange(0.4, 1, 0.02)
beta = numpy.arange(0.01,  0.5, 0.02)

# define number of searches within b and beta (for histograms later)
b_bins = (1-0.4)/0.05
beta_bins = 0.5/0.005

# this calculates the number of b/beta unique combinations
bbeta = np.array(list(itertools.product(b, beta)))
bbeta_pair_no = int((bbeta[:,1].shape)[0])
print(bbeta_pair_no)

750


***
### *Run Regression*


In [9]:
n = 768 # this is the number of samples taken each time (ideally the same as the size of the dataset)
resamples = 1000 # this is the number of bootstrap resamples

In [10]:
#here we initiate the variables
r_maximum = np.ones((resamples))*np.nan
b_maxr = np.ones((resamples))*np.nan
beta_maxr = np.ones((resamples))*np.nan
slope_max = np.ones((resamples))*np.nan
intercept_max = np.ones((resamples))*np.nan
random_Fic = np.ones((resamples,n))*np.nan
random_Foc = np.ones((resamples,n))*np.nan
random_Fsi = np.ones((resamples,n))*np.nan
random_depth = np.ones((resamples, n))*np.nan

In [ ]:
# here we run through the function which finds the best fit of b and beta of the range given
# we do this as many times as given in the resamples (usually at least as many data points)

for i in np.arange(0,resamples,1):
    w = time.time()
    r_maximum[i], b_maxr[i], beta_maxr[i], slope_max[i], intercept_max[i] = \
    JG_BBCael(forjoesouth,bbeta,n,bbeta_pair_no)
    if i == 1:
        w1 = time.time()
        print(i, ':   ', w1-w)
    if i == 100:
        w2=time.time()
        print(i, ':    ', w2-w1)
    if i == 500:
        w3=time.time()
        print(i, ':    ', w3-w2)
    if i == 999:
        w4=time.time()
        print(i, ':    ', w4-w3)

/gpfs/home/mep22dku/.conda/envs/swamp2/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/gpfs/scratch/mep22dku/CaelJG-intercomparison/JGFig1-ocean_opal_ballasting/scripts/major_axis_regression.py:125: RuntimeWarning: invalid value encountered in subtract
  xp = _x - xm
/gpfs/home/mep22dku/.conda/envs/swamp2/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/gpfs/home/mep22dku/.conda/envs/swamp2/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: All-NaN slice encountered
/gpfs/home/mep22dku/.conda/envs/swamp2/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: Mean of empty slice
/gpfs/home/mep22dku/.conda/envs/swamp2/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice
/gpfs/home/mep22dku/.conda/envs/swamp2/lib/python3.7/site-packages/ipykern

1 :    0.21254229545593262
100 :     24.778528690338135
500 :     171.67560386657715


-------
# Plot Histograms
here we're looking to make sure that the searches for b and beta don't fall at the edges of the search limits

In [ ]:
plt.hist(b_maxr, range=(0.00, 1), bins=int(b_bins))

In [ ]:
plt.hist(beta_maxr, range=(0.02, 2), bins=int(beta_bins))

-----
# Create median values for plotting graphs

In [ ]:
# find median values of the best fit regressions

r_median = np.nanmedian(r_maximum)
b_median = np.nanmedian(b_maxr)
beta_median = np.nanmedian(beta_maxr)
intercept_median = np.nanmedian(intercept_max)
slope_median = np.nanmedian(slope_max)

# Calculate alpha(si) and alpha(ic) values from the median beta, intercept and slope values
where kappa = e^intercept and gamma = slope of the line

y = Foc(z/1000)

x = Fic * beta(Fsi))^gamma

and: beta is ~ to a_si/a_ic

### therefore:

if: y = k * (beta * x^gamma)

beta = alpha_si/alpha_ic

alpha_ic = kappa^(1/gamma)

alpha_si = beta * kappa^(1/gamma)

k(kappa) = e^(intercept) and gamma = slope

the equation becomes Foc = (alpha_ic Fic + alpha_si Fsi)^gamma (z/1000)^b. so beta = alpha_si/alpha_ic.


In [ ]:
kappa = np.exp(intercept_max)
kappa_median = np.nanmedian(np.exp(intercept_max))

In [ ]:
a_ic_1 = kappa**(1/slope_median)
a_si_1 = beta_median*kappa**(1/slope_median)

In [ ]:
a_ic = kappa_median**(1/slope_median)
a_si = beta_median*kappa_median**(1/slope_median)

In [ ]:
print(a_ic)
print(a_si)

In [ ]:
print('r2:        ', r_median)
print('b:         ', b_median)
print('a_ic:      ', a_ic)
print('a_si:      ', a_si)
print('intercept: ', intercept_median)
print('slope:     ', slope_median)

-----
### Create values for plotting graphs

In [ ]:
forjoesouth.shape

In [ ]:
log_y = np.log(forjoesouth[:,3]) + (b_median * np.subtract(np.log(forjoesouth[:,0]),np.log(1000)))
log_x = np.log(kappa_median**slope_median*((forjoesouth[:,4]) + (beta_median * forjoesouth[:,5])))

y = np.exp(log_y)
x = np.exp(log_x)

scale = forjoesouth[:,4] / forjoesouth[:,5]


In [ ]:
x_line = [0.00001, np.max(x)+10000]
y_line = kappa_median*x_line**slope_median

In [ ]:
ax = plt.gca()
plt.plot(x_line, y_line, color='black', ls='-.', lw=2)
g = ax.scatter(x, y, c=scale, cmap=parula_map, norm=matplotlib.colors.LogNorm(vmin=0.01, vmax=100), s=40)
ax.set_yscale('log')
ax.set_xscale('log')
#plt.ylim(1e-2, 316.227766017)
#plt.xlim(0.00316227766, 600)
#axis([.005 500 .005 500])
plt.ylim(.005, 500)
plt.xlim(.005, 500)
cbar = plt.colorbar(g)

plt.title(r'$Southern \ Ocean \ (r^{2} = 0.89)$', size=16)

plt.ylabel(r'$F_{oc}\left ({z}  \right )^{0.67} \ (mg \ m^{-2} \ d^{-1})$', size=12) 

plt.xlabel(r'$(1.15 \times F_{ic}) + (0.195 \times F_{si})$', size=12)

cbar.set_label(r'${F_{ic}} \ / \ {F_{si}}$', size=14)

plt.show()